# Ampliación de la atención de una sola cabeza a la atención multicabecera

Ampliacion de la clase de atención casual a una de atención multicabezera.

El  término  "multicabezal"  se  refiere  a  la  división  del  mecanismo  de  atención  en  múltiples  "cabezas",  cada  una  operando  de  forma  independiente.  En  este  contexto,  un  único  módulo  de  atención  causal  puede  considerarse  atención  de  cabeza  única,  donde  solo  hay  un  conjunto  de  ponderaciones  de  atención  que  procesan  la  entrada  secuencialmente.

## Ailamiento de múltiples capas de atención de un solo cabezal

En  términos  prácticos,  implementar  la  atención  de  múltiples  cabezas  implica  crear  múltiples  instancias del  mecanismo  de  autoatención cada uno con sus pesos y luego combinar sus resultados.  Usar  múltiples  instancias  del  mecanismo  de  autoatención  puede  ser  computacionalmente  intensivo,  pero  es  crucial  para  el  tipo  de reconocimiento  de  patrones  complejos  por  los  que  son  conocidos  modelos  como  los  LLM  basados  en  transformadores.


![Texto alternativo](./imgs/3.23.png)

- En **atención de una sola cabeza** usas una sola matriz \( W_v \) para calcular los valores → un único vector de contexto \( Z \).

- En **atención de múltiples cabezas (multi-head)**, se usan varias matrices \( W_{v1}, W_{v2}, \dots \) (y lo mismo para \( W_q, W_k \)).

- Así, obtienes varios vectores de contexto (\( Z_1, Z_2, \dots \)) que luego se concatenan para formar un vector final.

La  idea  principal  detrás  de  la  atención  de  múltiples  cabezas  es  ejecutar  el  mecanismo  de  atención  varias  veces  (en  paralelo)  con  diferentes  proyecciones  lineales  aprendidas:  los  resultados  de  multiplicar  los  datos  de  entrada  (como  los  vectores  de  consulta,  clave  y  valor  en  los  
mecanismos  de  atención)  por  una  matriz  de  peso.
En  el  código,  podemos  lograr  esto  implementando  una  clase  MultiHeadAttentionWrapper  simple  que  apila  múltiples  instancias  de  nuestro  CausalAttention  implementado  previamente.módulo.

In [16]:
import torch
import torch.nn as nn

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     
    [0.55, 0.87, 0.66], # journey  (x^2)
    [0.57, 0.85, 0.64], # starts   (x^3)
    [0.22, 0.58, 0.33], # with     
    [0.77, 0.25, 0.10], # one      
    [0.05, 0.80, 0.55]] # step     
)

d_in = 3
d_out = 2

batch = torch.stack((inputs, inputs), dim=0)

class CasualAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout,  qkv_bias=False):
        super().__init__()
        self.out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.dropout = nn.Dropout(dropout)  #Capa de abandono

        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length),diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape                            
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.transpose(1, 2)              #transponer las dimensiones 1 y 2,manteniendo la dimension del lote en la primera dimension (0) 
        attn_scores.masked_fill_(                                 #las operaciones con guion dinal sen realiza en el lugar lo que evita copias de meoria inecesarias
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf) 
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec

In [17]:
class MiltiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CasualAttention(d_in, d_out, context_length, dropout) for _ in range(num_heads)]
        )
    
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

Por  ejemplo,  si  utilizamos  esta  clase  MultiHeadAttentionWrapper  con  dos  cabezas  de  atención  (a  través  de  num_heads=2)  y  la  dimensión  de  salida  CausalAttention  d_out=2,  esto  da  como  resultado  vectores  de  contexto  de  4  dimensiones  (d_out*num_heads=4)


![Texto alternativo](./imgs/3.24.png)

In [18]:
torch.manual_seed(123)
context_length = batch.shape[1]
mha = MiltiHeadAttentionWrapper(d_in, 2, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


## Implementación de la atención multicabecera con división de peso

Se usa MultiHeadAttentionWrapper  para  implementar  la  atención  multicabezal  mediante  la  superposición  de  varios  módulos  de  atención  de  un  solo  cabezal.  Esto  se  
logró  instanciando  y  combinando  varios  objetos  CausalAttention .

En  lugar  de  mantener  dos  clases  separadas,  MultiHeadAttentionWrapper  y  CausalAttention,  podemos  
combinar  ambos conceptos en uno solo. 

En  MultiHeadAttentionWrapper,  se  implementan  múltiples  cabezales  mediante  la  creación  de  una  lista  de  objetos  CausalAttention  (self.heads),  cada  uno  de  los  cuales  representa  un  cabezal  de  atención  independiente. La  clase  CausalAttention  ejecuta  el  mecanismo  de  atención  de  forma  independiente,  y  los  resultados  de  cada  
encabezado  se  concatenan.  Por  el  contrario,  la  clase  MultiHeadAttention ,  que  se  describe  a  continuación ,  integra  la  funcionalidad  multiencabezado  en  una  sola  clase.  Divide  la  entrada  en  varios  encabezados  mediante  la  remodelación  de  los  tensores  proyectados  de  consulta,  clave  y  valor,  y  luego  combina  los  resultados  de  
estos  encabezados  tras  calcular  la  atención.

In [19]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads                        #Reduzca  la  atenuación  de  la  proyección  para  que  coincida  con  la  atenuación  de  salida  deseada
        #si d_out=512 y num_heads=8, cada cabeza tendrá dimensión 64
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)                   #Capa lineal para combinar las salidas de la cabeza
        self.dropout = nn.Dropout(dropout)
        self.register_buffer( #evitar mirar a tokens futuros
            'mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
    def forward(self, x):
        b, num_tokens, d_in = x.shape #(2, 6, 3)
        keys = self.W_key(x)                                      #(b, num_tokens, d_out)
        queries = self.W_query(x)                                 
        values = self.W_value(x)                                  
        #Dividir  implícitamente  la  matriz  añadiendo  una  dimensión  `num_heads`.  Luego,  desenrollamos  la  última  dimensión:  (b,
        #núm_tokens,  d_out)  >  (b,  núm_tokens,  núm_cabezas,  cabeza_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim) 
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        keys = keys.transpose(1, 2)  #Transponer  de  la  forma  (b,  num_tokens,  num_heads,  head_dim)  a  (b,  num_heads,  num_tokens,  head_dim)                             
        queries = queries.transpose(1, 2)                         
        values = values.transpose(1, 2)  

        attn_scores = queries @ keys.transpose(2, 3)            
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]   #Mascara de truncamiento 
        attn_scores.masked_fill_(mask_bool, -torch.inf)           
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)       
        #Combina  cabezas,  donde  self.d_out  =  self.num_heads  *  self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)         #Agregar  una  proyección  lineal  opcional         
        return context_vec

A  nivel  general,  en  el  MultiHeadAttentionWrapper  anterior,  apilamoscomoelmúltiples  capas  de  atención  de  una  sola  cabeza  que  combinamos  en  una  capa  de  atención  de  múltiples  cabezas.La  clase  MultiHeadAttention  adopta  un  enfoque  integrado.  Comienza  con  un  multihead capa  y  luego  divide  internamente  esta  capa  en  cabezas  de  atención  individuales.

![Texto alternativo](./imgs/3.25.png)

En la atención multi-cabeza, los tensores de consultas (Q), claves (K) y valores (V) primero se generan con capas lineales de dimensión (b, num_tokens, d_out).
Luego:

1. División de la dimensión de salida

- Se reordena con .view en (b, num_tokens, num_heads, head_dim),

- donde head_dim = d_out / num_heads.

- Esto separa la información en varias cabezas más pequeñas.

2. Transposición

- Se aplica .transpose para obtener (b, num_heads, num_tokens, head_dim).

- Este paso pone la dimensión num_heads delante de num_tokens, lo que facilita los cálculos de atención en paralelo.

3. Importancia

- La división y transposición permiten alinear correctamente Q, K y V por cabeza y realizar las multiplicaciones de matrices de atención de forma eficiente en lotes (batch matrix multiplication).

In [20]:
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],      # (b,num_heads,num_tokens,head_dim)=(1, 2, 3, 4)       
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                    [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])
print(a@a.transpose(2, 3)) #(3*4) @ (4*3) = (3*3)
a.transpose(2, 3).shape

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


torch.Size([1, 2, 4, 3])

In [21]:
first_head = a[0, 0, :, :]
print(first_head)
first_res = first_head @ first_head.T
print(first_res)

tensor([[0.2745, 0.6584, 0.2775, 0.8573],
        [0.8993, 0.0390, 0.9268, 0.7388],
        [0.7179, 0.7058, 0.9156, 0.4340]])
tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])


Después de calcular los pesos de atención y los vectores de contexto:

1. Reorganización de los vectores de contexto

- Se transponen nuevamente a (b, num_tokens, num_heads, head_dim).

- Luego se remodelan (aplanan) a (b, num_tokens, d_out), combinando las salidas de todas las cabezas.

2. Proyección de salida

- Se aplica una capa lineal self.out_proj tras combinar las cabezas.

- Esta capa no es estrictamente necesaria, pero se usa en muchas arquitecturas LLM para mezclar la información final.

3. Eficiencia

- La implementación de MultiHeadAttention es más eficiente que la de MultiHeadAttentionWrapper.

- Solo se realiza una multiplicación de matrices por Q, K y V, en lugar de repetirla para cada elemento de atención, reduciendo el costo computacional.

4. Uso

- Se puede usar de manera similar a SelfAttention o CausalAttention en arquitecturas de transformers.

In [22]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape) #la dimension de salida esta controlada por d_out

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


A modo de comparación:

- GPT-2 pequeño: 12 cabezas, vector de contexto de 768.

- GPT-2 grande: 25 cabezas, vector de contexto de 1600.

En los modelos GPT, los tamaños de incrustación de los tokens y del contexto coinciden (d_in = d_out).